# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [34]:
from pyspark.sql import SparkSession

In [35]:
spark = SparkSession.builder.appName('churn').getOrCreate()

In [36]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [37]:
new_data = spark.read.csv('new_customers.csv',inferSchema=True,header=True)

In [38]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [39]:
new_data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [40]:
data.head(1)[0]

Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)

In [41]:
data.na.drop()
new_data.na.drop()

DataFrame[Names: string, Age: double, Total_Purchase: double, Account_Manager: int, Years: double, Num_Sites: double, Onboard_date: timestamp, Location: string, Company: string]

In [42]:
from pyspark.sql.functions import year
data = data.withColumn('Year',year(data['Onboard_date']))
new_data = new_data.withColumn('Year',year(new_data['Onboard_date']))

In [43]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn',
 'Year']

In [44]:
from pyspark.ml.feature import VectorAssembler

In [45]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Years',
 'Num_Sites',
 'Year'],outputCol='features')

In [46]:
output = assembler.transform(data)

In [47]:
new_output = assembler.transform(new_data)

In [48]:
final_data = output.select('features','Churn')

In [49]:
new_final_data = output.select('features')

In [50]:
from pyspark.ml.classification import LogisticRegression

In [51]:
# logreg = LogisticRegression(labelCol='Churn').setMaxIter(200).setTol(0.01)
logreg = LogisticRegression(labelCol='Churn')

In [52]:
final_data.describe().show()

+-------+-------------------+
|summary|              Churn|
+-------+-------------------+
|  count|                900|
|   mean|0.16666666666666666|
| stddev| 0.3728852122772358|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [53]:
model = logreg.fit(final_data)

In [54]:
predictions = model.transform(new_final_data)

In [56]:
predictions.printSchema()

root
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)



In [57]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [58]:
eval_model = logreg.fit(train_data)

In [59]:
eval_results = eval_model.transform(test_data)

In [60]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [61]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')

In [62]:
auc = my_eval.evaluate(eval_results)

In [63]:
print(auc)

0.5
